# Отток клиентов

## Введение

### Задание

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

### Описание данных

- **RowNumber** — индекс строки в данных
- **CustomerId** — уникальный идентификатор клиента
- **Surname** — фамилия
- **CreditScore** — кредитный рейтинг
- **Geography** — страна проживания
- **Gender** — пол
- **Age** — возраст
- **Tenure** — количество недвижимости у клиента
- **Balance** — баланс на счёте
- **NumOfProducts** — количество продуктов банка, используемых клиентом
- **HasCrCard** — наличие кредитной карты
- **IsActiveMember** — активность клиента
- **EstimatedSalary** — предполагаемая зарплата

Целевой признак
- **Exited** — факт ухода клиента

## Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder

## Подготовка данных

In [2]:
data = pd.read_csv('/datasets/Churn.csv')

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


### Вывод 

- Поле **Tenure** содержит пустые значения 
- Поля **Surname**, **Geography**, **Gender** не числовые
- Предположительно поля **RowNumber**, **CustomerId**, **Surname**, **Geography**, **Gender** не влиют на результат 

## Кодирование признаков 

Поля Surname, Geography, Gender — являются категориальными признаками и мы решаем задачу классификации, поэтому нам подойдет кодировка OrdinalEncoder

In [22]:
encoder = OrdinalEncoder()

In [23]:
tmp = data[['Surname', 'Geography', 'Gender']]
tmp =  pd.DataFrame(encoder.fit_transform(tmp), columns=tmp.columns)

In [25]:
data[['Surname', 'Geography', 'Gender']].head(10)

,Surname,Geography,Gender
0,Hargrave,France,Female
1,Hill,Spain,Female
2,Onio,France,Female
3,Boni,France,Female
4,Mitchell,Spain,Female
5,Chu,Spain,Male
6,Bartlett,France,Male
7,Obinna,Germany,Female
8,He,France,Male
9,H?,France,Male


In [26]:
tmp.head(10)

,Surname,Geography,Gender
0,1115.0,0.0,0.0
1,1177.0,2.0,0.0
2,2040.0,0.0,0.0
3,289.0,0.0,0.0
4,1822.0,2.0,0.0
5,537.0,2.0,1.0
6,177.0,0.0,1.0
7,2000.0,1.0,0.0
8,1146.0,0.0,1.0
9,1081.0,0.0,1.0


In [28]:
data[['Surname', 'Geography', 'Gender']] = tmp

In [30]:
data.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,1115.0,619,0.0,0.0,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,1177.0,608,2.0,0.0,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,2040.0,502,0.0,0.0,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,289.0,699,0.0,0.0,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,1822.0,850,2.0,0.0,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,537.0,645,2.0,1.0,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,177.0,822,0.0,1.0,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,2000.0,376,1.0,0.0,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,1146.0,501,0.0,1.0,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,1081.0,684,0.0,1.0,27,2.0,134603.88,1,1,1,71725.73,0


## Исследование задачи

## Борьба с дисбалансом

## Тестирование модели

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: задача исследована
    - [ ]  Исследован баланс классов
    - [ ]  Изучены модели без учёта дисбаланса
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 3: учтён дисбаланс
    - [ ]  Применено несколько способов борьбы с дисбалансом
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 4: проведено тестирование
- [ ]  Удалось достичь *F1*-меры не менее 0.59
- [ ]  Исследована метрика *AUC-ROC*